# Digit Recognizer

## Download the datasets

The data should be downloaded from these links and placed in the `./datasets` folder, named as indicated:

- [train.csv](https://www.kaggle.com/c/digit-recognizer/download/train.csv) | as `digit_train.csv`
- [test.csv](https://www.kaggle.com/c/digit-recognizer/download/test.csv) | as `digit_test.csv`

## Load the dataset

In [1]:
import csv
import numpy as np
from keras.utils.np_utils import to_categorical

def get_batch(filename, n=100):
    """Return a batch of training data."""
    with open(filename, 'r') as csvfile:
        next(csvfile) # Skip header
        datareader = csv.reader(csvfile)
        batch = list()
        count = 0
        for row in datareader:
            if count < n:
                batch.append(row)
                count += 1
            else:
                yield batch
                batch = [row]
                count = 1
    if count != 0:
        yield batch
        
def train_batch(n=100):
    while True:
        for batch in get_batch('./datasets/digit_train.csv', n):
            batch = np.array(batch)
            data = batch[:,1:] 
            labels = to_categorical(batch[:,0], 10)
            yield data, labels

def test_batch(n=100):
    for batch in get_batch('./datasets/digit_test.csv', n):
        yield np.array(batch)

Using TensorFlow backend.


In [2]:
def save_result(filename, labels):
    with open('output/{}.csv'.format(filename), 'w') as fd:
        print('ImageId,Label', file=fd)
        for i, label in enumerate(labels):
            print("{},{}".format(i + 1, label), file=fd)

## KerasModel

Loads the needed packages

In [61]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Reshape, Flatten, Dropout
from keras.layers.normalization import BatchNormalization

Prepare the model

In [1]:
model = Sequential()
model.add(Reshape((28, 28, 1), input_shape=(784,)))
model.add(Convolution2D(32, 5, 5, border_mode='same', activation='sigmoid'))
model.add(MaxPooling2D())
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(64, 5, 5, border_mode='same', activation='sigmoid'))
model.add(MaxPooling2D())
model.add(BatchNormalization(axis=1))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.summary()


Automatic pdb calling has been turned ON


NameError: name 'Sequential' is not defined

> <ipython-input-1-caac11b1b1da>(2)<module>()
      1 get_ipython().magic('pdb')
----> 2 model = Sequential()
      3 model.add(Reshape((28, 28, 1), input_shape=(784,)))
      4 model.add(Convolution2D(32, 5, 5, border_mode='same', activation='sigmoid'))
      5 model.add(MaxPooling2D())

ipdb> ?

Documented commands (type help <topic>):
EOF    c          d        h         next    pp       retval  u          whatis
a      cl         debug    help      p       psource  run     unalias    where 
alias  clear      disable  ignore    pdef    q        rv      undisplay
args   commands   display  interact  pdoc    quit     s       unt      
b      condition  down     j         pfile   r        source  until    
break  cont       enable   jump      pinfo   restart  step    up       
bt     continue   exit     n         pinfo2  return   tbreak  w        

Miscellaneous help topics:
exec  pdb

Undocumented commands:
l  list  ll  longlist

ipdb> p
*** SyntaxError: unexpected EOF while parsing
i

In [ ]:
%%bash
for i in {1..5}
do
    echo 

In [63]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Train the model

In [ ]:
model.fit_generator(train_batch(), samples_per_epoch=42000, nb_epoch=8)

Epoch 1/8
42000/42000 [==============================] - 176s - loss: 0.5322 - acc: 0.9052      

In [65]:
labels = list()
for data in test_batch():
    labels += list(model.predict_classes(data, batch_size=100))
    
save_result('digit_keras', labels)

100/100 [==============================] - 0s


## Tensorflow Softmax Regression Model

In [11]:
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

y = tf.matmul(x, W) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for data, labels in train_batch():
    train_step.run(feed_dict={x: data, y_: labels})

In [14]:
prediction = tf.argmax(y, 1)
labels = list()
for data in test_batch():
    labels += list(prediction.eval(feed_dict={x: data}))
    
save_result('digit_tf', labels)

## Tensorflow Convolutional Neural Network


Train the model

In [15]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])  
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

for i, (data, labels) in enumerate(train_batch()):
    if i%50 == 0:
        train_accuracy = accuracy.eval(feed_dict={ x: data, y_: labels, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: data, y_: labels, keep_prob: 0.5})

step 0, training accuracy 0.11
step 50, training accuracy 0.78
step 100, training accuracy 0.9
step 150, training accuracy 0.91
step 200, training accuracy 0.93
step 250, training accuracy 0.9
step 300, training accuracy 0.93
step 350, training accuracy 0.97
step 400, training accuracy 0.91


Make the prediction on test set

In [16]:
prediction = tf.argmax(y_conv, 1)
labels = list()
for data in test_batch():
    labels += list(prediction.eval(feed_dict={x: data, keep_prob: 1.0}))
    
save_result('digit_cnn', labels)